First part:

To download ANI-1 database and generate .ac files by antechamber

In [ ]:
import h5py
import numpy as np
import os, re
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
import openbabel as ob
from openbabel import pybel as pb
from multiprocessing import Pool
RDLogger.DisableLog('rdApp.*') #hiding the warning messages

In [ ]:
os.system('wget -c https://figshare.com/ndownloader/files/9057631 -O ANI1_release.tar.gz')
os.system('tar -xf ANI1_release.tar.gz')
# os.system('rm ANI1_release.tar.gz')

In [ ]:
os.system('mkdir frcmod GAFF_ac2 pdb')
current_path = os.path.abspath('.')
    
nb_cpu = 30
for i in range(nb_cpu):
    os.system('mkdir tmp' + str(i))

max_conf = 10 # Sometimes the .xyz -> .mdl -> .ac fails due to antechamber and it may take too much to try all conforamtion.
                # You can set max_conf=10 to try the first 10 conformations and it will save a lot of time.
                # If you set max_conf=1e20, it will try all possible conformations.

In [ ]:
# Create the .ac files for all molecules. To save time, only consider ten conformations
# It is possible to clear the current output of notebook! 
def para_process(m):
    os.chdir(current_path + '/tmp' + str(m))
    new_all_key = all_key[m*nb_processing:(m+1)*nb_processing]
    for i, key in enumerate(new_all_key):
        os.system('rm *')
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        species = f['gdb11_s0'+s_num][key]['species'][:]
        coord = f['gdb11_s0'+s_num][key]['coordinates'][:]
        num_atoms = len(species)
        for p in range(min(max_conf, len(coord))):
            if os.path.isfile('tmp.ac'):
                break
            openfile = open('tmp.xyz', 'w+')
            openfile.write(str(num_atoms) + ' \n')
            openfile.write('0 1 ' + key + '\n')
            for j in range(num_atoms):
                openfile.write(species[j].decode() + '     '+str(coord[p,j,0])+'    '+str(coord[p,j,1])+'    '+str(coord[p,j,2])+'\n')
            openfile.close()
            os.system('obabel -ixyz tmp.xyz -omdl -O tmp.mdl')
            if not os.system('antechamber -i tmp.mdl -fi mdl -o tmp.ac -fo ac -pf y -c bcc -at gaff2'):
                os.system('cp tmp.ac ../GAFF_ac2/' + name + '.ac')
    return m

for s in range(8):
    s_num = str(s+1)
    f = h5py.File('./ANI-1_release/ani_gdb_s0'+s_num+'.h5','r')
    all_key = []
    for key in f['gdb11_s0'+s_num].keys():
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        if not os.path.exists(current_path + '/GAFF_ac2/' + name + '.ac'):
            all_key.append(key)
    nb_processing = int(len(all_key) / nb_cpu) + 1
    with Pool(processes=nb_cpu) as pool:
        for results in pool.imap(para_process,range(nb_cpu)):
            pass
os.chdir(current_path)

In [ ]:
# If the molecule cannot give good .ac file by .mdl file, we can only try .pdb file 
def para_process(m):
    os.chdir(current_path + '/tmp' + str(m))
    new_all_key = all_key[m*nb_processing:(m+1)*nb_processing]
    for i, key in enumerate(new_all_key):
        os.system('rm *')
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        species = f['gdb11_s0'+s_num][key]['species'][:]
        coord = f['gdb11_s0'+s_num][key]['coordinates'][:]
        num_atoms = len(species)
        for p in range(min(max_conf, len(coord))):
            if os.path.isfile('tmp.ac'):
                break
            openfile = open('tmp.xyz', 'w+')
            openfile.write(str(num_atoms) + ' \n')
            openfile.write('0 1 ' + key + '\n')
            for j in range(num_atoms):
                openfile.write(species[j].decode() + '     '+str(coord[p,j,0])+'    '+str(coord[p,j,1])+'    '+str(coord[p,j,2])+'\n')
            openfile.close()
            os.system('obabel -ixyz tmp.xyz -opdb -O tmp.pdb')        
            if not os.system('antechamber -i tmp.pdb -fi pdb -o tmp.ac -fo ac -pf y -c bcc -at gaff2'):
                os.system('cp tmp.ac ../GAFF_ac2/' + name + '.ac')
    return m

for s in range(8):
    s_num = str(s+1)
    f = h5py.File('./ANI-1_release/ani_gdb_s0'+s_num+'.h5','r')
    all_key = []
    for key in f['gdb11_s0'+s_num].keys():
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        if not os.path.exists(current_path + '/GAFF_ac2/' + name + '.ac'):
            all_key.append(key)
    nb_processing = int(len(all_key) / nb_cpu) + 1
    with Pool(processes=nb_cpu) as pool:
        for results in pool.imap(para_process,range(nb_cpu)):
            pass
os.chdir(current_path)

In [ ]:
# generate the good .pdb file
def para_process(m):
    os.chdir(current_path + '/tmp' + str(m))
    new_all_key = all_key[m*nb_processing:(m+1)*nb_processing]
    for i, key in enumerate(new_all_key):
        os.system('rm *')
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        species = f['gdb11_s0'+s_num][key]['species'][:]
        coord = f['gdb11_s0'+s_num][key]['coordinates'][:]
        smi = ''
        for j in f['gdb11_s0'+s_num][key]['smiles'][:]:
            smi += j.decode()
        num_atoms = len(species)
        bool_ok = False
        for p in range(min(max_conf, len(coord))):
            if os.path.isfile('tmp.ac'):
                break
            openfile = open('tmp.xyz', 'w+')
            openfile.write(str(num_atoms) + ' \n')
            openfile.write('0 1 ' + key + '\n')
            for j in range(num_atoms):
                openfile.write(species[j].decode() + '     '+str(coord[p,j,0])+'    '+str(coord[p,j,1])+'    '+str(coord[p,j,2])+'\n')
            openfile.close()
            os.system('obabel -ixyz tmp.xyz -opdb -O tmp.pdb')        
            TmpMol = AllChem.MolFromPDBFile('tmp.pdb',removeHs=False)
            template = AllChem.MolFromSmiles(smi, sanitize=True)
            template = Chem.AddHs(template)
            try:
                AllChem.AssignBondOrdersFromTemplate(template, TmpMol)
                os.system('cp tmp.pdb ../pdb/'+name+'.pdb')
                bool_ok = True
            except:
                pass
            if bool_ok:
                break

    return m

for s in range(8):
    s_num = str(s+1)
    f = h5py.File('./ANI-1_release/ani_gdb_s0'+s_num+'.h5','r')
    all_key = []
    for key in f['gdb11_s0'+s_num].keys():
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        if not os.path.exists(current_path + '/pdb/' + name + '.pdb'):
            all_key.append(key)
    nb_processing = int(len(all_key) / nb_cpu) + 1
    with Pool(processes=nb_cpu) as pool:
        for results in pool.imap(para_process,range(nb_cpu)):
            pass
os.chdir(current_path)

In [ ]:
# create the complementary GAFF parameter files by parmchk2
def para_process(m):
    os.chdir(current_path + '/tmp' + str(m))
    new_all_name = all_name[m*nb_processing:(m+1)*nb_processing]
    for i, name in enumerate(new_all_name):
        os.system('rm *')
        if not os.system('parmchk2 -i ../GAFF_ac2/' + name + '.ac -f ac -o tmp'):
            os.system('mv tmp ../frcmod/' + name)
    return m

all_name = []
for root, dirs, files in os.walk(current_path + '/GAFF_ac2'):
    for file in files:
        all_name.append(re.split(r'(.ac)',file)[0])
nb_processing = int(len(all_name) / nb_cpu) + 1
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(para_process,range(nb_cpu)):
        pass
os.chdir(current_path)

In [ ]:
# delete the tmp directory
os.system('rm -rf tmp*')

Second part:

To generate 'mol.txt' file. 

You should restart the kernel!

In [ ]:
import os
import re
import random
import numpy as np
import functools
from multiprocessing import Pool
import h5py
# calculate the molecules can/cannot be processed

nb_cpu=30
current_path = os.path.abspath('.')
success_key = []
fail_key = []
for s in range(5):
    s_num = str(s+1)
    f = h5py.File('./ANI-1_release/ani_gdb_s0'+s_num+'.h5','r')
    for key in f['gdb11_s0'+s_num].keys():
        num = re.split(r'(-)',key)[-1]
        name = 's' + s_num + '_' + num
        if os.path.exists(current_path + '/pdb/' + name + '.pdb') and os.path.exists(current_path + '/GAFF_ac2/' + name + '.ac'):
            success_key.append(name)
        else:
            fail_key.append(name)
print(len(success_key), 'molecules are ok!')
print(len(fail_key), 'molecules are failed!')

os.chdir(current_path + '/../../')
from src.ForceFields.GAFF.ReadPara import read_GAFF2_dat
from src.ANI1.ReadMol import para_process_mol

mol = []
nb_time_processing = 20
GAFF = read_GAFF2_dat(False, False)
with Pool(processes=nb_cpu) as pool:
    for results in pool.imap(functools.partial(para_process_mol, all_name=success_key, dataset='ANI-1', GAFF=GAFF, bond_morse=False, angle_ub=False, max_len=26, nb_time_processing=nb_time_processing),range(int(np.ceil(len(success_key)/nb_time_processing)))):
        i, mol_part = results
        mol += mol_part
        if (i+1) % (10*nb_cpu) == 0:
            print(i+1,'/',int(np.ceil(len(all_name)/nb_time_processing)),' finished')
print('Molecule processing finished!')


In [ ]:
# generate 'mol.txt', which contains all process molecule names
random.shuffle(mol) 
mol_name = []
for i in range(len(mol)):
    mol_name.append(mol[i].mol_name)
write_file = open(current_path+'/mol.txt', 'w+')
for i in range(len(mol_name)):
    write_file.write(mol_name[i]+'\n')
write_file.close()